In [ ]:
from dataclasses import dataclass
from typing import Union, Any

Level = Union[int, str]

class RichTypeError(TypeError):
    pass

@dataclass(frozen=True, slots=True)
class Arg:
    name: str

@dataclass(frozen=True, slots=True)
class Val:
    value: Any

def do_shit(level):
    raise RichTypeError(
        "Expected", Arg("level"), "to be", Level, "given", type(level),
        Val(level)
    )

In [25]:
from numbers import Number
from typing import Annotated, Optional, get_args, get_origin, get_type_hints
from splatlog.lib.typeguard import satisfies
from typeguard import check_type
import dataclasses

@dataclasses.dataclass(frozen=True)
class ValueRange:
    min: Optional[Number] = None
    max: Optional[Number] = None
    
    def is_valid(self, value: Number) -> bool:
        if self.min is not None and value < self.min:
            return False
        if self.max is not None and value > self.max:
            return False
        return True

Verbosity = Annotated[int, ValueRange(min=0)]

args = get_args(Verbosity)

def my_check_type(name, value, expected_type):
    check_type(name, value, expected_type)

    if get_origin(expected_type) is Annotated:
        for arg in get_args(expected_type):
            if isinstance(arg, ValueRange) and not arg.is_valid(value):
                raise TypeError(
                    "type of {} must be {}; got {} instead".format(
                        name,
                        expected_type,
                        type(value)
                    )
                )



my_check_type("verbosity", -1, Verbosity)

TypeError: type of verbosity must be typing.Annotated[int, ValueRange(min=0, max=None)]; got <class 'int'> instead

In [31]:
from collections import defaultdict

def tree_default_factory():
    return defaultdict(tree_default_factory)

def tree_build(entries):
    tree = defaultdict(tree_default_factory)
    for (*key_path, final_key), value in entries:
        target = tree
        for key in key_path:
            target = target[key]
        target[final_key] = value
    return tree

tree_build((
    (("a", "b"), 1),
    (("a", "c"), 2),
    (("a", "d", "e"), 3),
))

defaultdict(<function __main__.tree_default_factory()>,
            {'a': defaultdict(<function __main__.tree_default_factory()>,
                         {'b': 1,
                          'c': 2,
                          'd': defaultdict(<function __main__.tree_default_factory()>,
                                      {'e': 3})})})

In [28]:
from pprint import pprint

class Blah:
    def __getitem__(self, *key):
        pprint(key)

b = Blah()

b[1, 2, 3]

((1, 2, 3),)


In [ ]:
from collections.abc import Generator
from splatlog.lib.collections import deep_each

iterable: list[int] = [1, 2, 3]
blah = deep_each(iterable)
value = next(blah)

In [7]:
from collections.abc import Iterable

isinstance(bytearray(), Iterable)

True